In [1]:
# Imports
import pandas as pd  # type: ignore
import base64
from utils import (
    get_rules_list,
    create_train_test_split,
    create_model,
    payload_to_vec,
    create_adv_train_test_set,
)
from modsec import init_modsec

from sklearn.ensemble import RandomForestClassifier  # type: ignore
from wafamole.evasion import EvasionEngine  # type: ignore

In [2]:
# Set up variables

attack_data_path = "data/attacks_20k.sql"
sane_data_path = "data/sanes_20k.sql"

rule_ids = get_rules_list()
modsec = init_modsec()

In [3]:
# Create train and test sets and train model

paranoia_level = 4

# train, test = create_train_test_split(
#     attack_file=attack_data_path,
#     sane_file=sane_data_path,
#     train_attacks_size=2500,  # paper uses 10000
#     train_sanes_size=2500,  # paper uses 10000
#     test_attacks_size=500,  # paper uses 2000
#     test_sanes_size=500,  # paper uses 2000
#     modsec=modsec,
#     rule_ids=rule_ids,
#     paranoia_level=paranoia_level,
# )
# train.to_csv("data/train_5k.csv", index=False)
# test.to_csv("data/test_1k.csv", index=False)

# load the train and test sets from disk
train = pd.read_csv("data/train_5k.csv")
test = pd.read_csv("data/test_1k.csv")

wafamole_model, threshold = create_model(
    train=train,
    test=test,
    model=RandomForestClassifier(n_estimators=160, random_state=666),
    desired_fpr=0.01,
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Reading and parsing data...
Splitting into train and test...
Creating vectors...


Processing payloads: 100%|██████████| 1000/1000 [00:36<00:00, 27.06it/s]


Done!
Train shape: (5000, 3) | Test shape: (1000, 3)
Model trained successfully!
Evaluating model...
Default threshold: 0.5
              precision    recall  f1-score   support

      attack       0.97      0.94      0.95       500
        sane       0.94      0.97      0.95       500

    accuracy                           0.95      1000
   macro avg       0.95      0.95      0.95      1000
weighted avg       0.95      0.95      0.95      1000

Adjusted threshold: 0.8027458911979671
              precision    recall  f1-score   support

      attack       0.85      0.99      0.92       500
        sane       0.99      0.83      0.90       500

    accuracy                           0.91      1000
   macro avg       0.92      0.91      0.91      1000
weighted avg       0.92      0.91      0.91      1000



In [4]:
# adversarial training

engine = EvasionEngine(wafamole_model)
engine_settings = {
    "max_rounds": 200,
    "round_size": 10,
    "timeout": 10,
    "threshold": threshold,
}

train_adv, test_adv = create_adv_train_test_set(
    train=train,
    test=test,
    train_adv_size=50,
    test_adv_size=50,
    engine=engine,
    engine_settings=engine_settings,
)

print(f"train_adv looks like: {train_adv[:5]}")
print(f"test_adv looks like: {test_adv[:5]}")

Optimizing train payloads...


100%|██████████| 50/50 [03:14<00:00,  3.89s/it]


Optimizing test payloads...


100%|██████████| 50/50 [03:03<00:00,  3.67s/it]

train_adv looks like:                                                    data   label
2779  REVsZVRFIGZyb20gYFRhYmAgV0hFcmUgYENPTDNgIDwgLT...    sane
2610  VVBEQVRFIGB0YWJgIFNFVCBgY29sM2AgPSAweDMgV0hFUk...  attack
1120  U2VMRUN0IGBjT0wxYCBGUm9tIGBUYUJgIHdIRXJFIGBjb0...  attack
662   VVBEQVRFDGB0YWJgIFNFVCBgY29sMmAgPSA0IFdIRVJFIG...  attack
4452  VVBEQVRFIGB0YWJgIFNFVCBgY29sM2AgPSAwIFdIRVJFIG...    sane
test_adv looks like:                                                   data   label
487  REVMRVRFIEZST00gYHRhYmAgV0hFUkUgYGNvbDNgID0gJ2...  attack
642  SU5TRVJUIElOVE8gYHRhYmAgKCBgY29sMmAsIGBjb2wyYC...  attack
521  VVBEQVRFIGB0YWJgIFNFVCBgY29sMmAgPSAzIFdIRVJFIG...  attack
242  REVMRXRlIEZSb20gYFRBQmAgd0hFcmUgYGNvbDJgID0gJ3...    sane
739  VVBEQVRFIGB0YWJgIFNFVCBgY29sMWAgTElLRSA3IFdIRV...    sane


In [5]:
# Test the model

payload = 'SELECT SLEEP(5)#";'  # attack

payload_base64 = base64.b64encode(payload.encode("utf-8")).decode("utf-8")
vec = payload_to_vec(payload_base64, rule_ids, modsec, paranoia_level)
is_attack = wafamole_model.classify(payload)
print(f"Payload: {payload}")
print(f"Vec: {vec}")
print(f"Confidence: {round(is_attack, 5)}")

Payload: SELECT SLEEP(5)#";
Vec: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
Confidence: 0.96247


In [6]:
# Try and evade the WAF with WAFamole

# TODO: decide if we want to just evade (reach threshold) or if we want to minimize confidence until convergence
# Probably the latter, but we need to decide on a stopping criterion (time out probably)
min_confidence, min_payload = engine.evaluate(
    payload=payload,
    max_rounds=200,
    round_size=10,
    timeout=60,
    threshold=0.0,
)
print()
print(f"Min payload: {min_payload.encode('utf-8')}")
print(f"Min confidence: {round(min_confidence, 5)}")
print(
    f"Reduced confidence from {round(is_attack, 5)} to {round(min_confidence, 5)} (reduction of {round(is_attack - min_confidence, 5)})"
)

print("\nEvasion successful" if min_confidence < threshold else "Evasion failed")

[!] Execution timed out
Reached confidence 0.6891363316546412
with payload
SELECT SLEEP((SELECT (SELECT 0x5)))#";v

Min payload: b'SELECT\xc2\xa0SLEEP((SELECT (SELECT 0x5)))#";v\x0b'
Min confidence: 0.68914
Reduced confidence from 0.96247 to 0.68914 (reduction of 0.27333)

Evasion successful
